# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 1, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4D', 'insta_score': 0.114, 'expected_score': -18, 'adjustment': 6},
 {'call': '3D', 'insta_score': 0.347, 'expected_score': -37, 'adjustment': 17},
 {'call': 'PASS',
  'insta_score': 0.173,
  'expected_score': -59,
  'adjustment': 9},
 {'call': '2N', 'insta_score': 0.156, 'expected_score': -75, 'adjustment': 8},
 {'call': '5D', 'insta_score': 0.136, 'expected_score': -279, 'adjustment': 7}]

In [4]:
bid.samples

['J.J8x.A98xxx.Txx A8x.A9xxx.JT.Q8x Qx.KQTx.xxx.AKJx KT9xxxx.x.KQ.9xx 0.73409 \n 1H-1N-2S-3D-P-P-3S-4D-P-P-P (8.8)  \n 1H-1N-2S-P-P-X-P-3D-3S-P-P-4D-P-P-P (8.8)  \n 1H-1N-2S-2N-P-P-P (5.3)  \n 1H-1N-2S-5D-P-P-X-P-P-P (8.8)  \n 1H-1N-2S-4D-P-P-P (8.8) ',
 'J.J8x.A98xxx.Txx Kxx.KT9xx.x.AJ9x Qx.AQxx.KJx.K8xx AT98xxx.x.QTx.Qx 0.70590 \n 1H-1N-2S-3D-3S-4D-4S-5D-P-P-X-P-P-P (9.0)  \n 1H-1N-2S-P-P-X-P-3D-3S-P-P-4D-P-P-P (9.0)  \n 1H-1N-2S-2N-P-P-P (5.0)  \n 1H-1N-2S-5D-P-P-X-P-P-P (9.0)  \n 1H-1N-2S-4D-P-P-P (9.0) ',
 'J.J8x.A98xxx.Txx KTx.QT9xx.x.AJxx 8x.AKxx.QJTx.KQx AQ9xxxx.x.Kx.98x 0.62963 \n 1H-1N-2S-3D-P-P-3S-4D-P-P-4S-P-P-P (9.6)  \n 1H-1N-2S-P-P-X-P-3D-3S-P-P-4D-P-P-P (9.9)  \n 1H-1N-2S-2N-P-P-3S-4D-P-P-4S-P-P-P (9.6)  \n 1H-1N-2S-5D-P-P-X-P-P-P (9.9)  \n 1H-1N-2S-4D-P-P-4S-P-P-P (9.6) ',
 'J.J8x.A98xxx.Txx 9.AQT9xx.KT.Q8xx K8xx.Kx.QJx.AK9x AQTxxxx.xx.xx.Jx 0.61171 \n 1H-1N-2S-3D-3H-P-4H-P-P-P (8.2)  \n 1H-1N-2S-P-3H-P-4H-P-P-P (8.2)  \n 1H-1N-2S-2N-P-P-P (7.0)  \n 1H-1N-2S-5D-P-P-X-P

In [5]:
auction = ["PAD_START", "PAD_START", "1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.559,
  'expected_score': 342,
  'adjustment': 279},
 {'call': '5D',
  'insta_score': 0.354,
  'expected_score': -1948,
  'adjustment': 177}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [6]:
bid.samples

['J.J8x.A98xxx.Txx xxx.AKQTx.xx.K8x AK.xxx.KQJx.Q9xx QT98xxx.9x.T.AJx 0.31800 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (2.1)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (4.6) ',
 'J.J8x.A98xxx.Txx AQx.AT9xx.xx.98x Tx.KQx.KJTx.AKxx K98xxxx.xx.Q.QJx 0.31280 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (4.0)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (2.0) ',
 'J.J8x.A98xxx.Txx 9xx.AQTxx.Kx.AJx Ax.K9xx.QJTx.KQx KQT8xxx.x.x.98xx 0.30995 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (2.8)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (3.0) ',
 'J.J8x.A98xxx.Txx Kxx.AKxxx.Jx.98x Ax.QTx.KQTxx.AJx QT98xxx.9x..KQxx 0.30279 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (2.8)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (2.9) ',
 'J.J8x.A98xxx.Txx Kxx.Kxxxx.xx.A9x T8x.AQ9.KQJx.KJx AQ9xxx.Tx.T.Q8xx 0.29800 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (4.2)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (2.7) ',
 'J.J8x.A98xxx.Txx T9x.AKTxxx.J.Axx KQ.Q9xx.QTx.KQJ8 A8xxxxx..Kxx.9xx 0.29507 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (2.6)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (4.7) ',
 'J.J8x.A98xxx.Txx A9x.K9xxx.Q.Q8xx xx.AQTx.KJxx.AKx KQT8x

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

J.J8x.A98xxx.Txx 8xxx.9xx.Qx.J8xx AKQ9xx.AQ.x.AKQx Tx.KTxxx.KJTx.9x
J.J8x.A98xxx.Txx 98xx.KT9x.Qx.K9x AKQxx.AQ.Tx.AQJx Txx.xxxx.KJx.8xx
J.J8x.A98xxx.Txx 9xxx.QTxxx.Qx.Qx AKQ8xx.AKx.x.AK9 Tx.9x.KJTx.J8xxx
J.J8x.A98xxx.Txx T9x.QTxx.KQ.K9xx AKQ8xx.AK9.J.AJx xxx.xxx.Txxx.Q8x
J.J8x.A98xxx.Txx Kx.Txxx.Txx.K9xx AQT98x.AKQ9.J.AJ xxxx.xx.KQx.Q8xx
J.J8x.A98xxx.Txx 8xxx.Txx.KJT.AJx AKQ9x.AKQx.Qx.Q9 Txx.9xx.xx.K8xxx
J.J8x.A98xxx.Txx xxx.Txx.Kxx.A9xx AKTxx.AKQ9.J.KQx Q98x.xxx.QTx.J8x
J.J8x.A98xxx.Txx Kx.KQT9xx.KTx.9x AQxxxx.A.QJ.AKQx T98x.xxx.xx.J8xx
J.J8x.A98xxx.Txx Q9x.Q9xxx.xx.9xx ATxxx.AKT.KJ.AQJ K8xx.xx.QTx.K8xx
J.J8x.A98xxx.Txx 8xx.Kxxx.Jx.KJ8x AKQxxx.AQx.Kx.AQ T9x.T9x.QTx.9xxx
